In [2]:
import utils.spacetrack as strack
import utils.time as tu
import requests
import json
from datetime import datetime, timedelta
import time

In [ ]:
raise Exception("Are you sure you want to be running this notebook? Really it should only be ran one time. If yes you can comment this out")

## Establish Connection to space-track.org API

You need an account to access this data programmatically, as they track who is using their API. They **will** ban your account (speaking from personal experience) if you query them too many times in a row, so it's important to access the API carefully when making multiple calls in succession.

My login information is stored in a file, `./ted-login.ini`, which **will not** be committed to the repository. 

You shouldn't have to run this notebook, but if you do want to pull data see `strack_example.ini` for instructions on how to fill out your username/password file. 

**Be sure to double-check that your login information is not accidentally committed.**

In [2]:
CREDENTIALS_NEVER_PRINT = strack.loadSiteCred("./utils/ted-strack.ini")

In [3]:
with requests.Session() as session:
    login_resp = strack.api_login(session, CREDENTIALS_NEVER_PRINT)
    print("you logged in!")
    session.close()
print("you logged out!")

you logged in!
you logged out!


## Pull Catalog Snapshots

We need to do the following:

1. Establish a start date.
2. Establish an end date.
3. Write code that accesses the space-track.org API (without violating its rules) and pulls data.
4. Save each day's data as its own text file.

**Note:** This code should really only be run one time, so I'm not too worried about optimizing or multi-threading.


In [ ]:

start_string = '2023-03-15'
stop_string = '2024-03-15'
n_pulls = 0

with requests.Session() as session:
    login_resp = strack.api_login(session, CREDENTIALS_NEVER_PRINT)

    start_date = datetime.strptime(start_string, '%Y-%m-%d')
    end_date = datetime.strptime(stop_string, '%Y-%m-%d')

    current_date = start_date

    while current_date <= end_date:
        # Only process if it's the 1st or 15th of the month
        if current_date.day == 1 or current_date.day == 15:
            print(f"Making request for {current_date}")
        
            next_date = current_date + timedelta(days=1)
            
            res = strack.api_catalog_snapshot(session, tu.timestamp_query(current_date), tu.timestamp_query(next_date))
            
            if res.status_code == 200:
                with open(f"../data/raw/snapshots/snapshot_states_{tu.timestampe_file(current_date)}.txt", 'w') as file:
                    file.write(res.text)
                print(f"Saved snapshot for {current_date}")
            else:
                print(f"Failed to get data for {current_date}, status code: {res.status_code}")
            
            n_pulls += 1

            if n_pulls > 10:
                print("Snoozing for 60 secs for rate limit reasons (max 20/min and 200/hr)...")
                time.sleep(60)
                n_pulls = 0

        current_date += timedelta(days=1)

    session.close()

print("done!")


Making request for 2023-03-15 00:00:00
Saved snapshot for 2023-03-15 00:00:00
Making request for 2023-04-01 00:00:00
Saved snapshot for 2023-04-01 00:00:00
Making request for 2023-04-15 00:00:00
Saved snapshot for 2023-04-15 00:00:00
Making request for 2023-05-01 00:00:00
Saved snapshot for 2023-05-01 00:00:00
Making request for 2023-05-15 00:00:00
Saved snapshot for 2023-05-15 00:00:00
Making request for 2023-06-01 00:00:00
Saved snapshot for 2023-06-01 00:00:00
Making request for 2023-06-15 00:00:00
Saved snapshot for 2023-06-15 00:00:00
Making request for 2023-07-01 00:00:00
Saved snapshot for 2023-07-01 00:00:00
Making request for 2023-07-15 00:00:00
Saved snapshot for 2023-07-15 00:00:00
Making request for 2023-08-01 00:00:00
Saved snapshot for 2023-08-01 00:00:00
Making request for 2023-08-15 00:00:00
Saved snapshot for 2023-08-15 00:00:00
Snoozing for 60 secs for rate limit reasons (max 20/min and 200/hr)...
Making request for 2023-09-01 00:00:00
Saved snapshot for 2023-09-01 00

## Satellite Catalogs: Decayed and Not Decayed

We are also interested in getting state catalog information. Our objects should be in orbit, but we need to account for any objects that have decayed. Therefore, we have two catalogs: the active one and the decayed one.


In [4]:
new_catalogs = strack.request_repull_current_and_decayed_satcats("./utils/ted-strack.ini")

with open('../data/raw/current_satcat_'+tu.timestamp_file_now()+'.json', 'w') as json_file:
    json.dump(json.loads(new_catalogs[0]), json_file)

with open('../data/raw/decayed_satcat_'+tu.timestamp_file_now()+'.json', 'w') as json_file:
     json.dump(json.loads(new_catalogs[1]), json_file)